# Introduction

In [1]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import Entrez
from Bio import SeqIO

## Partie A

In [2]:
Entrez.email = 'mathieu@colmon.fr'

# On fait une fonction pour récupérer le premier
# résultat d'une requête sous forme d'un objet Seq
def getSeq(request):
  # On exécute la requête demandée
  searchReq = Entrez.esearch(db='nucleotide', term=request)
  # On lit le résultat
  searchRes = Entrez.read(searchReq)
  # On ferme la requête
  searchReq.close()

  # On demande le premier résultat
  firstReq = Entrez.efetch(db='nucleotide', id=searchRes['IdList'], rettype='gb')
  # On lit le résultat
  firstRes = SeqIO.read(firstReq, 'gb')
  # On ferme la requête
  firstReq.close()

  return firstRes

# On traite chaque requête
SEQs = [
  getSeq('SARS-Cov2 [ORGN] AND srcdb_refseq [PROP]'),
  getSeq('SARSr-CoV RaTG13'),
  getSeq('Pangolin coronavirus isolate MP789 MT121216'),
]

# On écrit le résultat dans un fichier
SeqIO.write(SEQs, './sequences/seq_covid.gb', 'genbank')

3

## Partie B

In [3]:
SEQs = SeqIO.parse('./sequences/seq_covid.gb', 'genbank')

for seq in SEQs:
  for f in seq.features:
    print(f)

type: source
location: [0:29903](+)
qualifiers:
    Key: collection_date, Value: ['Dec-2019']
    Key: country, Value: ['China']
    Key: db_xref, Value: ['taxon:2697049']
    Key: host, Value: ['Homo sapiens']
    Key: isolate, Value: ['Wuhan-Hu-1']
    Key: mol_type, Value: ['genomic RNA']
    Key: organism, Value: ['Severe acute respiratory syndrome coronavirus 2']

type: 5'UTR
location: [0:265](+)
qualifiers:

type: gene
location: [265:21555](+)
qualifiers:
    Key: db_xref, Value: ['GeneID:43740578']
    Key: gene, Value: ['ORF1ab']
    Key: locus_tag, Value: ['GU280_gp01']

type: CDS
location: join{[265:13468](+), [13467:21555](+)}
qualifiers:
    Key: codon_start, Value: ['1']
    Key: db_xref, Value: ['GeneID:43740578']
    Key: gene, Value: ['ORF1ab']
    Key: locus_tag, Value: ['GU280_gp01']
    Key: note, Value: ['pp1ab; translated by -1 ribosomal frameshift']
    Key: product, Value: ['ORF1ab polyprotein']
    Key: protein_id, Value: ['YP_009724389.1']
    Key: ribosomal_sl

## Partie C

In [4]:
# On importe le module permettant d'aligner les séquences
from Bio.Align.Applications import MafftCommandline

In [5]:
# On Aligne les séquences du fichier 'spike.fasta'
myStdout = MafftCommandline(input='./sequences/spike.fasta')()

# On écrit le résultat dans 'aln-spike.fasta'
with open('./sequences/aln-spike.fasta', 'w') as w: w.write(myStdout)

ApplicationError: Non-zero return code 1 from 'mafft ./sequences/spike.fasta', message "'mafft' n'est pas reconnu en tant que commande interne"